In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
df.head(5)

In [ ]:
# Plot 1: Weekly Sales Distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['Weekly_Sales'], bins=50, kde=True, color='blue')
plt.title('Distribution of Weekly Sales')
plt.xlabel('Weekly Sales')
plt.ylabel('Frequency')
plt.xlim(0, df['Weekly_Sales'].max())
plt.show()

In [ ]:
# Plot 1: Weekly Sales Distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['Weekly_Sales'], bins=50, kde=True, color='blue')
plt.title('Distribution of Weekly Sales')
plt.xlabel('Weekly Sales')
plt.ylabel('Frequency')
plt.xlim(0, 100000)
plt.show()

In [ ]:
# Plot 2: Sales by Store Type
plt.figure(figsize=(10, 6))
sns.boxplot(x='Type', y='Weekly_Sales', data=df)
plt.title('Weekly Sales by Store Type')
plt.xlabel('Store Type')
plt.ylabel('Weekly Sales')
plt.ylim(0, df['Weekly_Sales'].quantile(0.95))  # Exclude outliers
plt.show()

In [ ]:
# Plot 3: Correlation Heatmap
df_heatmap = df.drop(columns = ['Store', 'Dept'])
df_heatmap = df.select_dtypes(include=['number'])
plt.figure(figsize=(12, 8))
correlation_matrix = df_heatmap.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', square=True)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Plot 4: Sales Trends Over Time
df['Date'] = pd.to_datetime(df['Date'])
weekly_sales_over_time = df.groupby('Date')['Weekly_Sales'].sum().reset_index()

plt.figure(figsize=(14, 7))
sns.lineplot(x='Date', y='Weekly_Sales', data=weekly_sales_over_time)
plt.title('Total Weekly Sales Over Time')
plt.xticks(rotation=45)
plt.ylabel('Total Weekly Sales')
plt.show()

In [ ]:
outlier_columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5', 'Weekly_Sales']
df[outlier_columns].hist(bins=30, figsize=(15, 14))
plt.suptitle('Histograms of Outlier Variables', fontsize=20)
plt.show()

In [ ]:
df_clean = df

df_clean[outlier_columns].hist(bins=30, figsize=(15, 14))
plt.suptitle('Histograms of Outlier Variables', fontsize=20)
plt.show()

In [ ]:
# Plot 6: Markdowns Impact on Sales
markdown_columns = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
for markdown in markdown_columns:
    plt.figure(figsize=(10, 6))
    sns.boxplot(x=markdown, y='Weekly_Sales', data=df_clean)
    plt.title(f'Weekly Sales vs. {markdown}')
    plt.xlabel(markdown)
    plt.ylabel('Weekly Sales')
    plt.xlim(0, df_clean[markdown].max())  # Avoid too large xlim for better visualization
    plt.show()

In [ ]:
df_clean['Date'] = pd.to_datetime(df_clean['Date'])
df_clean_cpi = df_clean.groupby('Date')['CPI'].mean().reset_index()
plt.figure(figsize=(12,6))
plt.plot(df_clean_cpi['Date'], df_clean_cpi['CPI'], marker = 'o', linestyle = '-')
plt.title("CPI over time")
plt.xlabel("Date")
plt.ylabel("CPI")

plt.grid(True)
plt.show()

In [ ]:
#consolidate holiday columns
assert (df_clean['IsHoliday_x'] == df_clean['IsHoliday_y']).all(), "Holiday columns do not match"

df_clean.drop(columns='IsHoliday_y', inplace=True)
df_clean.rename(columns={'IsHoliday_x': 'IsHoliday'}, inplace=True)

In [ ]:
# from openai import OpenAI

# def feature_engineering(sample, target_variable):
#     client = OpenAI(
#         api_key="sk-REDACTED"
#     )

#     messages = [
#         {
#             "role": "system",
#             "content": "You are an expert data scientist. Suggest new features to improve prediction of the target variable."
#         },
#         {
#             "role": "user",
#             "content": f"""Here's a sample of my dataset (in CSV format):

# {sample}

# The target variable is: {target_variable}

# Please suggest 3 new features with:
# - A short name and explanation
# - Python code to create each one
# - Python code to visualize its relationship with the target variable
# """
#         }
#     ]

#     response = client.chat.completions.create(
#         model="gpt-4o-mini",
#         messages=messages
#     )

#     return response.choices[0].message.content

In [ ]:
# sample = df_clean.sample(n=100, random_state=1).to_csv(index=False)
# output = feature_engineering(sample, "Weekly_Sales")
# print(output)

In [ ]:
### 1. **Lagged Sales**
#This feature captures the weekly sales from previous periods, which can be indicative of trends or patterns in customer behavior. By looking back at sales from prior weeks, you can establish a baseline for expected performance.

df_clean['Date'] = pd.to_datetime(df_clean['Date'])
df_clean = df_clean.sort_values(by=['Store', 'Dept', 'Date'])
df_clean['Lagged_Sales'] = df_clean.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(1)

#to visualize
plt.figure(figsize=(12, 6))
sns.scatterplot(x='Lagged_Sales', y='Weekly_Sales', data=df_clean)
plt.title("Lagged Sales vs. Weekly Sales")
plt.xlabel("Lagged Sales")
plt.ylabel("Weekly Sales")
plt.show()

In [ ]:
df_clean.head(5)

In [ ]:
### **Sales Growth Rate**
### This feature calculates the percentage change in weekly sales compared to the previous week. A growing or declining trend can significantly affect future sales.

df_clean['Sales_Growth_Rate'] = round(df.groupby(['Store', 'Dept'])['Weekly_Sales'].pct_change() * 100, 2)

plt.figure(figsize=(12, 6))
sns.lineplot(x='Date', y='Sales_Growth_Rate', hue='Store', data=df_clean, errorbar=None)
plt.title("Sales Growth Rate Over Time by Store")
plt.xlabel("Date")
plt.ylabel("Sales Growth Rate (%)")
plt.show()

In [ ]:
### 3. **Temperature Z-score**
# This feature normalizes temperature observations into Z-scores, helping to identify how unusual temperatures may relate to weekly sales. Extreme temperatures can significantly impact customer footfall and sales.
from scipy.stats import zscore

df_clean['Temperature_Zscore'] = df_clean['Temperature'].transform(zscore)

plt.figure(figsize=(20, 10))
sns.boxplot(x='Temperature_Zscore', y='Weekly_Sales', data=df_clean)
plt.title("Temperature Z-score vs. Weekly Sales")
plt.xlabel("Temperature Z-score")
plt.ylabel("Weekly Sales")
plt.show()

In [ ]:
df_clean.isna().sum()

In [ ]:
df_clean['Sales_Growth_Rate'] = df_clean['Sales_Growth_Rate'].replace([np.inf, -np.inf], np.nan)
df_clean['Lagged_Sales'] = df_clean['Lagged_Sales'].fillna(df_clean['Lagged_Sales'].median())
df_clean['Sales_Growth_Rate'] = df_clean['Sales_Growth_Rate'].fillna(df_clean['Sales_Growth_Rate'].median())


print(df_clean['Lagged_Sales'].isna().sum())
print(df_clean['Sales_Growth_Rate'].isna().sum())

In [ ]:
df_clean.to_csv("df_clean.csv", index=False)